# Create date features

In [2]:
import os
import re
import pickle
import time
import datetime

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix, vstack

%matplotlib inline

# Custom modules
import const
import func

## Load data

In [3]:
print const.TRAIN_FILES
print const.TEST_FILES

['train_numeric', 'train_categorical_to_num', 'train_date']
['test_numeric', 'test_categorical_to_num', 'test_date']


In [4]:
lut = pd.read_csv(const.LOOK_UP_TABLE)
lut.set_index('name_dat', inplace=True)
lut.head(3)

,line,station,feature_nr,feat_nr_dat,name_cat,name_num,col_dat,col_num,col_cat,station_V2,line_V2
name_dat,,,,,,,,,,,
L0_S0_D1,0,0,0,1.0,NaN,L0_S0_F0,0.0,0.0,NaN,0.0,1.0
L0_S0_D3,0,0,2,3.0,NaN,L0_S0_F2,1.0,1.0,NaN,0.0,1.0
L0_S0_D5,0,0,4,5.0,NaN,L0_S0_F4,2.0,2.0,NaN,0.0,1.0


In [5]:
date_train = func.load_data_file(const.TRAIN_FILES[2])
date_test = func.load_data_file(const.TEST_FILES[2])

Returning <open file '/Volumes/My Book/kaggle_bosch/train_date.pkl', mode 'rb' at 0x115c48c90>.pkl
Returning <open file '/Volumes/My Book/kaggle_bosch/test_date.pkl', mode 'rb' at 0x115c48c90>.pkl


In [6]:
date_data = vstack([date_train['data']['features'],date_test['data']['features']], format='csr')
ids = pd.concat([date_train['data']['ids'], date_test['data']['ids']])
y = date_train['data']['y']

In [7]:
st_columns = lut.groupby('station_V2')['col_dat'].first().values
st_names = lut.groupby('station_V2')['col_dat'].first().index.values

In [8]:
date_data = pd.DataFrame(date_data[:, st_columns].todense()).replace(0, np.nan)
date_data.columns = [str(st_names[n]) for n in date_data.columns]

In [9]:
# Add clusters,response and id to data
# Add cluster info
cluster_info = pd.read_csv(os.path.join(const.DATA_PATH, 'eda_sample_clusters.csv'))
cluster_info.head(3)

,Id,unique_path,cluster_n8,cluster_n15,cluster_n25,cluster_n50,cluster_n150,cluster_n500
0,4,13409,1,2,3,47,36,369
1,6,7029,2,10,0,35,63,14
2,7,12763,1,2,19,3,132,477


In [10]:
date_data = date_data.merge(ids.reset_index(), left_index=True, right_index=True, how='left')

In [11]:
date_data = date_data.merge(cluster_info, left_on='Id', right_on='Id', how='left')

In [12]:
date_data = date_data.merge(y, left_on='Id', right_index=True, how='left')

In [13]:
# Get rid of decimals in timestamp
date_data = (100*date_data).round(0)

In [14]:
n_samples = date_data.shape[0]

In [15]:
print date_data.shape
date_data.head(3)

(2367495, 138)


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,index,Id,unique_path,cluster_n8,cluster_n15,cluster_n25,cluster_n50,cluster_n150,cluster_n500,Response
0,8224.0,8224.0,8224.0,NaN,8226.0,NaN,NaN,8226.0,8227.0,NaN,...,0,400,1340900,100,200,300,4700,3600,36900,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100,600,702900,200,1000,0,3500,6300,1400,0.0
2,161870.0,161870.0,161870.0,NaN,NaN,161872.0,161872.0,NaN,161873.0,NaN,...,200,700,1276300,100,200,1900,300,13200,47700,0.0


### Create features based on station

In [16]:
d_cols = date_data.columns[:128]

In [17]:
d_feat = pd.DataFrame(ids.Id.values, columns=['Id'])

In [18]:
# Time at station
d_feat = pd.concat([d_feat, date_data[d_cols]], axis=1)

In [19]:
d_feat.columns = ['Id'] + ['t_' + c for c in d_feat.columns if c!='Id']

In [20]:
d_feat.head()

,Id,t_0.0,t_1.0,t_2.0,t_3.0,t_4.0,t_5.0,t_6.0,t_7.0,t_8.0,...,t_42.0,t_43.0,t_44.0,t_45.0,t_46.0,t_47.0,t_48.0,t_49.0,t_50.0,t_51.0
0,4,8224.0,8224.0,8224.0,NaN,8226.0,NaN,NaN,8226.0,8227.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,161870.0,161870.0,161870.0,NaN,NaN,161872.0,161872.0,NaN,161873.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,114920.0,114920.0,114921.0,NaN,114922.0,NaN,NaN,114922.0,114922.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,60264.0,60264.0,NaN,60264.0,60266.0,NaN,NaN,60267.0,60267.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Delay at station
d_date = date_data.loc[:,d_cols].fillna(method='ffill', axis=1).diff(axis=1)

# Replace 0.0 by nan if station was not passed
d_date[date_data[d_cols].isnull()] = np.nan

# Change columns
d_date.columns = ['d_' + c for c in d_date.columns]

d_feat = pd.concat([d_feat, d_date], axis=1)

In [22]:
d_date.shape

(2367495, 128)

In [23]:
# Delay relative to cluster median (rel)
df_delta_devn8_rel = d_date.copy()
for cl in date_data['cluster_n50'].unique():
        df_delta_devn8_rel.loc[date_data['cluster_n50']==cl, :] /= \
                df_delta_devn8_rel.loc[date_data['cluster_n50']==cl, :].median()



In [24]:
df_delta_devn8_rel[df_delta_devn8_rel==np.inf] = 0
df_delta_devn8_rel.columns = ['d_rel_' + c for c in df_delta_devn8_rel.columns]

In [25]:
d_feat['ID'] = ids.Id.values

In [26]:
df_delta_devn8_rel['ID'] = ids.Id.values

In [27]:
d_feat.shape

(2367495, 258)

In [28]:
df_delta_devn8_rel.shape

(2367495, 129)

In [29]:
# Put ID column first
cols = [df_delta_devn8_rel.columns[-1]] + list(df_delta_devn8_rel.columns[:-1])
df_delta_devn8_rel = df_delta_devn8_rel[cols]

In [30]:
cols = [d_feat.columns[-1]] + list(d_feat.columns[:-1])
d_feat = d_feat[cols]

In [31]:
d_feat.drop('Id', axis=1, inplace=True)

In [32]:
d_date.head()

,d_0.0,d_1.0,d_2.0,d_3.0,d_4.0,d_5.0,d_6.0,d_7.0,d_8.0,d_9.0,...,d_42.0,d_43.0,d_44.0,d_45.0,d_46.0,d_47.0,d_48.0,d_49.0,d_50.0,d_51.0
0,NaN,0.0,0.0,NaN,2.0,NaN,NaN,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.0,0.0,NaN,NaN,2.0,0.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0.0,1.0,NaN,1.0,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.0,NaN,0.0,2.0,NaN,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
d_feat.to_csv(os.path.join(const.DATA_PATH, 'feat_set_date_station.csv'), index=False)

In [33]:
df_delta_devn8_rel.head()

,ID,d_rel_d_0.0,d_rel_d_1.0,d_rel_d_2.0,d_rel_d_3.0,d_rel_d_4.0,d_rel_d_5.0,d_rel_d_6.0,d_rel_d_7.0,d_rel_d_8.0,...,d_rel_d_42.0,d_rel_d_43.0,d_rel_d_44.0,d_rel_d_45.0,d_rel_d_46.0,d_rel_d_47.0,d_rel_d_48.0,d_rel_d_49.0,d_rel_d_50.0,d_rel_d_51.0
0,4,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,0.0,NaN,0.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df_delta_devn8_rel.to_csv(os.path.join(const.DATA_PATH, 'feat_set_date_station_rel_n50.csv'), index=False)

In [ ]:
del d_feat, d_date

## Create features based on lineV2

In [23]:
lines = lut['line_V2'].unique()

In [41]:
d_feat = pd.DataFrame(ids.Id.values, columns=['Id'])

In [42]:
# Use different line definition for 3 and 4
blocks = {'1':('0.0','11.0'),
         '2':('12.0','23.0'),
         '3.1':('24.1', '24.111'),
         '3.2':('24.2', '24.211'),
         '3.3':('24.3', '24.311'),
         '3':('24.1', '28.0'),
         '4.1':('25.1', '25.11'),
         '4.2':('25.202', '25.21'),
         '4.3':('25.212', '25.22'),
         '4.4':('25.222', '25.23'),
         '4':('25.1', '28.0'),
         '5':('26.0', '28.0'),
         '6':('29.0', '38.0'),
         '7':('39.0', '51.0')}

df_min=pd.concat([date_data.loc[:, v[0]:v[1]].min(1) for k,v in blocks.iteritems()], axis=1)
df_max=pd.concat([date_data.loc[:, v[0]:v[1]].max(1) for k,v in blocks.iteritems()], axis=1)
df_delta = df_max - df_min
df_delta = df_delta.replace(0, np.nan)

# Change column names
df_min.columns =  ['tmin_' + k for k,v in blocks.iteritems()]
df_max.columns =  ['tmax_' + k for k,v in blocks.iteritems()]
df_delta.columns =  ['tdelta_' + k for k,v in blocks.iteritems()]

In [43]:
# Replace difference of line 24
df_delta.loc[~df_delta['tdelta_4'].isnull(),'tdelta_3'] = np.nan

# Ajdust total for sublines
df_delta.loc[:,'tdelta_3'] -= df_delta.loc[:,['tdelta_3.1','tdelta_3.2','tdelta_3.3']].fillna(0).sum(1)
df_delta.loc[:,'tdelta_4'] -= df_delta.loc[:,['tdelta_4.1','tdelta_4.2','tdelta_4.3','tdelta_4.4']].fillna(0).sum(1)

In [44]:
df_delta.describe()

,tdelta_3.2,tdelta_3.3,tdelta_3.1,tdelta_4.4,tdelta_4.1,tdelta_4.3,tdelta_4.2,tdelta_1,tdelta_3,tdelta_2,tdelta_5,tdelta_4,tdelta_7,tdelta_6
count,54436.000000,264245.000000,47447.000000,34696.000000,111050.000000,8399.000000,14119.000000,1.350022e+06,366051.000000,484556.000000,579.000000,167621.000000,120176.000000,2.242263e+06
mean,3.147219,3.460955,2.957110,2.047239,12.849266,2.217288,2.073872,4.746032e+00,2024.822342,4.431376,250.868739,2645.937448,4.640219,3.604536e+01
std,3.240327,3.406196,3.044942,2.047044,3.282380,1.856820,1.766308,2.871497e+01,1617.120107,28.086674,523.011796,2921.841775,75.831819,3.797536e+02
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000e+00
25%,2.000000,2.000000,1.000000,1.000000,12.000000,1.000000,1.000000,3.000000e+00,835.000000,3.000000,20.000000,1491.000000,2.000000,2.000000e+00
50%,3.000000,3.000000,2.000000,1.000000,12.000000,2.000000,1.000000,3.000000e+00,1794.000000,3.000000,55.000000,2416.000000,3.000000,3.000000e+00
75%,3.000000,4.000000,3.000000,2.000000,13.000000,2.000000,2.000000,4.000000e+00,3137.000000,4.000000,175.500000,3240.000000,4.000000,3.000000e+00
max,73.000000,101.000000,89.000000,37.000000,185.000000,28.000000,19.000000,8.346000e+03,32203.000000,12874.000000,4280.000000,37421.000000,21216.000000,2.117100e+04


In [45]:
# Delay relative to cluster median (abs)
df_delta_devn8_abs = df_delta.copy()
for cl in date_data['cluster_n50'].unique():
        df_delta_devn8_abs.loc[date_data['cluster_n50']==cl, :] -= \
                df_delta_devn8_abs.loc[date_data['cluster_n50']==cl, :].median()
df_delta_devn8_abs.columns =  ['tdeltadevabs_' + k for k,v in blocks.iteritems()]            
# Delay relative to cluster median (rel)
df_delta_devn8_rel = df_delta.copy()
for cl in date_data['cluster_n50'].unique():
        df_delta_devn8_rel.loc[date_data['cluster_n50']==cl, :] /= \
                df_delta_devn8_rel.loc[date_data['cluster_n50']==cl, :].median()
df_delta_devn8_rel.columns =  ['tdeltadevrel_' + k for k,v in blocks.iteritems()] 

In [46]:
df_delta_devn8_rel[df_delta_devn8_rel==np.inf] = 0

In [47]:
df_delta_devn8_rel.describe()

,tdeltadevrel_3.2,tdeltadevrel_3.3,tdeltadevrel_3.1,tdeltadevrel_4.4,tdeltadevrel_4.1,tdeltadevrel_4.3,tdeltadevrel_4.2,tdeltadevrel_1,tdeltadevrel_3,tdeltadevrel_2,tdeltadevrel_5,tdeltadevrel_4,tdeltadevrel_7,tdeltadevrel_6
count,54436.000000,264245.000000,47447.000000,34696.000000,111050.000000,8399.000000,14119.000000,1.350022e+06,364121.000000,484556.000000,579.000000,167621.000000,120176.000000,2.242263e+06
mean,1.095920,1.160621,1.475710,2.016440,1.070950,1.117395,2.038282,1.581136e+00,1.122709,1.477158,2.895282,1.173797,1.547572,1.202240e+01
std,1.370324,1.139386,1.522904,2.034238,0.273362,0.954100,1.755977,9.570636e+00,0.921799,9.362215,6.529057,1.428242,25.277279,1.266795e+02
min,0.333333,0.333333,0.333333,0.400000,0.083333,0.500000,0.500000,3.333333e-01,0.000000,0.333333,0.005935,0.000000,0.333333,2.500000e-01
25%,0.666667,0.666667,0.500000,1.000000,1.000000,0.500000,1.000000,1.000000e+00,0.476678,1.000000,0.418143,0.680932,0.666667,6.666667e-01
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,0.998862,1.000000,1.000000,1.000000,1.000000,1.000000e+00
75%,1.333333,1.333333,1.500000,2.000000,1.083333,1.000000,2.000000,1.333333e+00,1.716724,1.333333,2.273035,1.413826,1.333333,1.000000e+00
max,33.500000,33.666667,44.500000,37.000000,15.416667,14.000000,19.000000,2.782000e+03,18.317975,4291.333333,69.800000,77.600000,7072.000000,7.611000e+03


In [48]:
d_feat = pd.concat([d_feat, df_min, df_max, df_delta, df_delta_devn8_abs, df_delta_devn8_rel], axis=1)

In [49]:
d_feat.head()

,Id,tmin_3.2,tmin_3.3,tmin_3.1,tmin_4.4,tmin_4.1,tmin_4.3,tmin_4.2,tmin_1,tmin_3,...,tdeltadevrel_4.1,tdeltadevrel_4.3,tdeltadevrel_4.2,tdeltadevrel_1,tdeltadevrel_3,tdeltadevrel_2,tdeltadevrel_5,tdeltadevrel_4,tdeltadevrel_7,tdeltadevrel_6
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8224.0,NaN,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,1.000000
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.666667
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161870.0,NaN,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.666667
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114920.0,NaN,...,NaN,NaN,NaN,0.666667,NaN,NaN,NaN,NaN,NaN,1.333333
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60264.0,NaN,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.666667


In [50]:
d_feat.to_csv(os.path.join(const.DATA_PATH, 'feat_set_date_lineV2.csv'), index=False)

## Line V2 features compressed + all

In [33]:
def compress(df):
    
    df = df.copy()
    
    tag = df.columns[0].split('_')[0] + '_'
    
    # Merge second block (line 6 & 7) (100% coverage)
    df.loc[df[tag + '7'].isnull(),tag + '7'] = df.loc[df[tag + '7'].isnull(),tag + '6']
    #print df['tdelta_7'].isnull().sum().astype(float)/n_samples
    df.drop(tag + '6', axis=1, inplace=True)
    
    # Merge 3.1-3.3
    df.loc[df[tag + '3.1'].isnull(),tag + '3.1'] = df.loc[df[tag + '3.1'].isnull(),tag + '3.2']
    df.loc[df[tag + '3.1'].isnull(),tag + '3.1'] = df.loc[df[tag + '3.1'].isnull(),tag + '3.3']
    #print (~df['tdelta_3.1'].isnull()).sum().astype(float)/n_samples
    df.drop([tag + '3.2',tag + '3.3'], axis=1, inplace=True)
    
    # Merge 4.1-4.4
    df.loc[df[tag + '4.1'].isnull(),tag + '4.1'] = df.loc[df[tag + '4.1'].isnull(),tag + '4.2']
    df.loc[df[tag + '4.1'].isnull(),tag + '4.1'] = df.loc[df[tag + '4.1'].isnull(),tag + '4.3']
    df.loc[df[tag + '4.1'].isnull(),tag + '4.1'] = df.loc[df[tag + '4.1'].isnull(),tag + '4.4']
    #print (~df['tdelta_4.1'].isnull()).sum().astype(float)/n_samples
    df.drop([tag + '4.2',tag + '4.3',tag + '4.4'], axis=1, inplace=True)
    
    # Merge 1, 2, 3.1-3.3, 4.1-4.4
    df.loc[df[tag + '1'].isnull(),tag + '1'] = df.loc[df[tag + '1'].isnull(),tag + '2']
    df.loc[df[tag + '1'].isnull(),tag + '1'] = df.loc[df[tag + '1'].isnull(),tag + '3.1']
    df.loc[df[tag + '1'].isnull(),tag + '1'] = df.loc[df[tag + '1'].isnull(),tag + '4.1']
    #print (~df['tdelta_1'].isnull()).sum().astype(float)/n_samples
    df.drop([tag + '2', tag + '3.1', tag + '4.1'], axis=1, inplace=True)
    
    # Merge 3 and 4 (and drop 5)
    df.loc[df[tag + '3'].isnull(),tag + '3'] = df.loc[df[tag + '3'].isnull(),tag + '4']
    #print (~df['tdelta_3'].isnull()).sum().astype(float)/n_samples
    df.drop([tag + '4', tag + '5'], axis=1, inplace=True)
    
    df.columns = [tag + 'block1',tag + 'block1a',tag + 'block2']
    
    print df.isnull().sum(0).astype(float)/n_samples
    
    return df

In [34]:
d_feat = pd.DataFrame(ids.Id.values, columns=['Id'])
d_feat = pd.concat([d_feat, 
                    compress(df_min), 
                    compress(df_max), 
                    compress(df_delta), 
                    compress(df_delta_devn8_abs), 
                    compress(df_delta_devn8_rel)], 
                   axis=1)

tmin_block1     0.003277
tmin_block1a    0.679657
tmin_block2     0.000496
dtype: float64
tmax_block1     0.003277
tmax_block1a    0.679657
tmax_block2     0.000496
dtype: float64
tdelta_block1     0.004095
tdelta_block1a    0.774584
tdelta_block2     0.002136
dtype: float64
tdeltadevabs_block1     0.004095
tdeltadevabs_block1a    0.774584
tdeltadevabs_block2     0.002136
dtype: float64
tdeltadevrel_block1     0.004095
tdeltadevrel_block1a    0.775399
tdeltadevrel_block2     0.002136
dtype: float64


In [35]:
d_feat['tmax'] = date_data[d_cols].max(1)
d_feat['tmin'] = date_data[d_cols].min(1)
d_feat['tdelta'] = d_feat['tmax'] - d_feat['tmin']

# Delay relative to cluster median (rel)
d_feat['tdelta_devrel'] = d_feat['tdelta']
for cl in date_data['cluster_n50'].unique():
        d_feat.loc[date_data['cluster_n50']==cl, 'tdelta_devrel'] /= \
                d_feat.loc[date_data['cluster_n50']==cl, 'tdelta'].median()
            
# Delay relative to cluster median (abs)
d_feat['tdelta_devabs'] = d_feat['tdelta']
for cl in date_data['cluster_n50'].unique():
        d_feat.loc[date_data['cluster_n50']==cl, 'tdelta_devabs'] -= \
                d_feat.loc[date_data['cluster_n50']==cl, 'tdelta'].median()
        


In [36]:
for col in d_feat.columns[1:]:
    if col.find('rel')>0:
        d_feat[col] = (d_feat[col]*100).round(0)
    else:
        d_feat[col] = (d_feat[col]/100).round(0)

In [37]:
d_feat.head()

,Id,tmin_block1,tmin_block1a,tmin_block2,tmax_block1,tmax_block1a,tmax_block2,tdelta_block1,tdelta_block1a,tdelta_block2,...,tdeltadevabs_block1a,tdeltadevabs_block2,tdeltadevrel_block1,tdeltadevrel_block1a,tdeltadevrel_block2,tmax,tmin,tdelta,tdelta_devrel,tdelta_devabs
0,4,82.0,NaN,87.0,82.0,NaN,87.0,0.0,NaN,0.0,...,NaN,0.0,100.0,NaN,100.0,87.0,82.0,5.0,194.0,2.0
1,6,1313.0,NaN,1316.0,1313.0,NaN,1316.0,0.0,NaN,0.0,...,NaN,-0.0,100.0,NaN,67.0,1316.0,1313.0,3.0,114.0,0.0
2,7,1619.0,NaN,1624.0,1619.0,NaN,1624.0,0.0,NaN,0.0,...,NaN,-0.0,100.0,NaN,67.0,1624.0,1619.0,6.0,223.0,3.0
3,9,1149.0,NaN,1154.0,1149.0,NaN,1154.0,0.0,NaN,0.0,...,NaN,0.0,67.0,NaN,133.0,1154.0,1149.0,5.0,191.0,2.0
4,11,603.0,NaN,606.0,603.0,NaN,606.0,0.0,NaN,0.0,...,NaN,-0.0,100.0,NaN,67.0,606.0,603.0,3.0,131.0,1.0


In [38]:
d_feat.to_csv(os.path.join(const.DATA_PATH, 'feat_set_date_all_compressed.csv'), index=False)